In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,48.99,81,75,11.50,broken clouds
1,1,Cabo San Lucas,MX,22.8909,-109.9124,84.00,34,75,11.50,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.20,38,0,10.36,clear sky
3,3,Rio Grande,BR,-32.0350,-52.0986,71.60,83,90,20.71,overcast clouds
4,4,Filipesti,RO,46.7500,26.8833,43.00,52,0,9.22,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cabo San Lucas,MX,22.8909,-109.9124,84.00,34,75,11.50,broken clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.35,74,8,8.57,light rain
9,9,Vaini,TO,-21.2000,-175.2000,80.60,69,20,5.75,few clouds
16,16,Shujalpur,IN,23.4000,76.7167,81.70,22,53,4.21,broken clouds
21,21,Sao Filipe,CV,14.8961,-24.4956,76.03,67,0,8.88,clear sky
25,25,Atuona,PF,-9.8000,-139.0333,80.71,76,19,17.43,few clouds
28,28,Mahebourg,MU,-20.4081,57.7000,77.00,94,75,4.61,broken clouds
33,33,Butaritari,KI,3.0707,172.7902,80.94,78,31,9.71,light rain
34,34,Puerto Ayora,EC,-0.7393,-90.3518,80.60,74,40,9.22,scattered clouds
35,35,Namatanai,PG,-3.6667,152.4333,78.04,85,100,6.20,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                234
City                   234
Country                234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No Empty Rows

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,84.00,broken clouds,22.8909,-109.9124,
5,Rikitea,PF,78.35,light rain,-23.1203,-134.9692,
9,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
16,Shujalpur,IN,81.70,broken clouds,23.4000,76.7167,
21,Sao Filipe,CV,76.03,clear sky,14.8961,-24.4956,
25,Atuona,PF,80.71,few clouds,-9.8000,-139.0333,
28,Mahebourg,MU,77.00,broken clouds,-20.4081,57.7000,
33,Butaritari,KI,80.94,light rain,3.0707,172.7902,
34,Puerto Ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,
35,Namatanai,PG,78.04,light rain,-3.6667,152.4333,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # 7. Drop the rows where there is no Hotel Name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,84.00,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
5,Rikitea,PF,78.35,light rain,-23.1203,-134.9692,Pension Maro'i
9,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort
16,Shujalpur,IN,81.70,broken clouds,23.4000,76.7167,Krishna Sweets
21,Sao Filipe,CV,76.03,clear sky,14.8961,-24.4956,Tortuga B&B
...,...,...,...,...,...,...,...
710,Karaton,ID,78.01,few clouds,-6.3424,106.1045,Parkir Mobil Wisata CAS Water Park
711,Pindiga,NG,88.84,overcast clouds,9.9833,10.9333,Accommodation Center
712,San Policarpo,PH,77.47,overcast clouds,12.1791,125.5072,JM Lodge
714,Camargo,MX,86.25,overcast clouds,27.6667,-105.1667,Hotel Santa Fe


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))